In [ ]:
pip install finpie

In [ ]:
pip install yahoofinancials

In [ ]:
pip install yahoo_fin

In [1]:
from yahoofinancials import YahooFinancials

import yahoo_fin.stock_info as si
import finpie

import csv
import numpy as np
import pandas as pd

In [2]:
src6 = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/list_comp.csv'

comp_list = pd.read_csv(src6, header=None)

rows = []
for i in range(len(comp_list)):
    if len(comp_list.iloc[i, 0].split(':'))>=2:
        rows.append(comp_list.iloc[i, 0].split(':')[1])

In [3]:
rows.sort()

In [4]:
del rows[rows.index('2412')]
del rows[rows.index('386')]
del rows[rows.index('6758')]
del rows[rows.index('7203')]
del rows[rows.index('7267')]
del rows[rows.index('7751')]
del rows[rows.index('8591')]
del rows[rows.index('902')]
del rows[rows.index('A015760')]
del rows[rows.index('A017670')]
del rows[rows.index('ABBN')]
del rows[rows.index('BAM.A')]
del rows[rows.index('BF.B')]
del rows[rows.index('BP.')]
del rows[rows.index('DGE')]
del rows[rows.index('EMBR3')]
del rows[rows.index('ENI')]
del rows[rows.index('FEMSA UBD')]
del rows[rows.index('FLOW')]
del rows[rows.index('FME')]
del rows[rows.index('NAV')]
del rows[rows.index('NOKIA')]
del rows[rows.index('NOVO B')]
del rows[rows.index('RDSA')]
del rows[rows.index('SN.')]
del rows[rows.index('ULVR')]

In [ ]:
def get_yearly_fs(ticker):
    is_annual_si = si.get_income_statement(ticker, yearly=True)
    bs_annual_si = si.get_balance_sheet(ticker, yearly=True)
    cf_annual_si = si.get_cash_flow(ticker, yearly=True)

    fs_annual_si = pd.concat([is_annual_si, bs_annual_si, cf_annual_si], axis=0)
    
    fs_annual_si['name']=fs_annual_si.index
    fs_annual_si.drop_duplicates(inplace=True)
    fs_annual_si.drop(columns=['name'], inplace=True)

    fs_annual_si = fs_annual_si.T
    
    fs_annual_si['date'] = pd.to_datetime(fs_annual_si.index).strftime("%d %B, %Y")
    #fs_annual_si.index = pd.to_datetime(fs_annual_si.index).strftime("%d %B, %Y")
    
    fs = fs_annual_si.to_dict('list')
    
    return fs
    

def get_quarterly_fs(ticker):
    is_quarterly_si = si.get_income_statement(ticker, yearly=False)
    bs_quarterly_si = si.get_balance_sheet(ticker, yearly=False)
    cf_quarterly_si = si.get_cash_flow(ticker, yearly=False)
    
    fs_quarterly_si = pd.concat([is_quarterly_si, bs_quarterly_si, cf_quarterly_si], axis=0)
    
    fs_quarterly_si['name']=fs_quarterly_si.index
    fs_quarterly_si.drop_duplicates(inplace=True)
    fs_quarterly_si.drop(columns=['name'], inplace=True)

    fs_quarterly_si = fs_quarterly_si.T
    
    fs_quarterly_si['date'] = pd.to_datetime(fs_quarterly_si.index).strftime("%d %B, %Y")
    
    fs = fs_quarterly_si.to_dict('list')
    
    return fs

In [ ]:
DB_focus_yearly = {}
for ticker in rows:
    print(ticker)
    DB_focus_yearly[ticker] = get_yearly_fs(ticker)

In [ ]:
DB_focus_quarterly = {}
for ticker in rows_clean:
    print(ticker)
    DB_focus_quarterly[ticker] = get_quarterly_fs(ticker)

In [ ]:
#rows2=[]
#for row in rows:
#    rows2.append([row])

#file = open('list_comp_fs_yahoo.csv', 'w+', newline ='') 
#with file:     
#    write = csv.writer(file) 
#    write.writerows(rows_clean2)

In [ ]:
from openpyxl import load_workbook

In [ ]:
book = load_workbook('DB_focus_annual2.xlsx')
writer = pd.ExcelWriter('DB_focus_annual2.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

for ticker, df in DB_focus_yearly.items():
    df.index = pd.to_datetime(df.index).strftime("%d %B, %Y")
    df.to_excel(writer, sheet_name=ticker)

writer.save()

In [ ]:
book = load_workbook('DB_focus_quartal2.xlsx')
writer = pd.ExcelWriter('DB_focus_quartal2.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

for ticker, df in DB_focus_quarterly.items():
    df.index = pd.to_datetime(df.index).strftime("%d %B, %Y")
    df.to_excel(writer, sheet_name=ticker)

writer.save()

In [ ]:
import json

In [ ]:
a_file = open("DB_focus_annual2.json",'w')
json.dump(DB_focus_yearly, a_file)
a_file.close()

In [ ]:
a_file = open("DB_focus_quartal2.json",'w')
json.dump(DB_focus_quarterly, a_file)
a_file.close()

In [ ]:
pip install jupyter_dash

In [ ]:
pip install dash

In [ ]:
pip install dash_core_components

In [ ]:
pip install dash_html_components

In [ ]:
pip install dash_table

In [ ]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash_table import DataTable
from dash_table import FormatTemplate

from dash.dependencies import Input, Output

import plotly.express as px
from datetime import datetime

from yahoofinancials import YahooFinancials

import yahoo_fin.stock_info as si
import finpie

import csv
import numpy as np
import pandas as pd

src1 = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/list_comp_fs_yahoo.csv'
src2 = 'https://raw.githubusercontent.com//Bayan2019/TCC/main/Yahoo-Finance-3.png'
src3 = 'https://raw.githubusercontent.com//Bayan2019/TCC/main/Logo_Curve_BG-Blue.png'
src4 = 'https://raw.githubusercontent.com//Bayan2019/TCC/main/Logo_Curve_BG-Blue1.png'
src5 = 'https://raw.githubusercontent.com//Bayan2019/TCC/main/jc2.png'
src6 = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/list_comp.csv'

tickers1 = pd.Series(si.tickers_other())
tickers2 = pd.Series(si.tickers_nasdaq())

tickers = pd.concat([tickers1, tickers2], axis=0)
tickers.drop_duplicates(inplace=True)


def get_title(ticker):
    #title = si.get_company_info(ticker).loc['longBusinessSummary', 'Value']\
    #.split(',')[0].split(' is ')[0].split('.')[0]
    #title = YahooFinancials(ticker).get_stock_quote_type_data()[ticker]['longName']
    title = finpie.Fundamentals(ticker, 'yahoo', freq ='A').profile().iloc[0, 0]
    
    return title

def plot_style(paper_bg='black', plot_bg='rgb(82, 82, 82)', font='violet'):
    plot_style = {'paper_bgcolor':paper_bg, 'plot_bgcolor':plot_bg,
                               'font': {'color':font}}
    return plot_style

# Create the Dash app
app = JupyterDash(__name__)

app.layout = html.Div([
 html.Img(src=src5, style={'width':'300px', 'display':'inline-block'}),
 html.Img(src=src2, style={'width':'120px', 'display':'inline-block'}),   
 html.H1('Financial Statement', style={'color':'rgb(191, 62, 255)', 'display':'inline-block'}),
 html.Img(src=src2, style={'width':'120px', 'display':'inline-block'}),
 html.Img(src=src5, style={'width':'300px', 'display':'inline-block'}),
 html.Br(),   
 html.Div(
  children = [
  html.Div(
   children = [   
   html.H3('Ticker'),  
   dcc.Dropdown(id='ticker', options = [{'label': i, 'value':i} for i in tickers], 
                 style={'width':'150px', 'margin':'0 auto', 'color':'blue'})],
   style={'display':'inline-block', 'margin':'0px 15px 0px 15px'}),
  html.Div(
   children = [   
   html.H3('Period'),
   dcc.Dropdown(id='period', 
        options = [{'label':'annual', 'value':'annual'}, {'label':'quarterly', 'value':'quarterly'}], 
        style={'width':'100px', 'margin':'0 auto', 'color':'blue', 'verticalAlign':'top'})],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'}),
  html.Div(
   children = [  
   html.H3('y-axis'),
   dcc.Dropdown(id='column_y', 
                style={'width':'250px', 'margin':'0 auto', 'color':'blue', 'verticalAlign':'top'})],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'}),
  html.Div(
   children = [   
   html.H3('ColorMap'),
   dcc.Dropdown(id='column_c', style={'width':'250px', 'color':'blue'})],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'}),
  html.Div(
   children = [  
   html.H3('Radius'),
   dcc.Dropdown(id='column_r', 
                style={'width':'250px', 'margin':'0 auto', 'color':'blue', 'vertical-align':'top'})],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'})], 
  style={'width':'1400px', 'height':'120px', 'display':'inline-block', 'border':'1px solid black'}),
 html.Br(),
 html.Div(
  children = [
  html.Span(id='information', style={'width':'400px', 'display':'inline-block', 'vertical-align':'bottom'}), 
  html.Div(
   children = [    
   dcc.Graph(id='scatter_fig', style={'width':'950px'}),
   dcc.Graph(id='bar_fig1', style={'width':'950px'})],
   style={'width':'950px', 'display':'inline-block'})],
  style={'display':'inline-block', 'vertical-align':'center'}),
 dcc.Graph(id='bar_fig2'),
 html.Br(),
 html.Div(
  children = [
  html.Div(
   children = [
   html.H3('First Column'),
   dcc.Dropdown(id='column_1', style={'width':'250px', 'color':'blue'}),
   html.H3('Second Column'),
   dcc.Dropdown(id='column_2', style={'width':'250px', 'color':'blue'}),
   html.H3('Third Column'),
   dcc.Dropdown(id='column_3', style={'width':'250px', 'color':'blue'})],
   style={'width':'250px', 'height':'450px', 'display':'inline-block',
    'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
  html.Div(
   children = [   
   dcc.Graph(id='line_polar_fig')], 
   style={'width':'900px', 'height':'450px','display':'inline-block'})],
  style={'width':'1400px', 'height':'450px', 'display':'inline-block', 'border':'1px solid black'}),   
 html.Br(),
 html.Img(src=src4, style={'width':'150px', 'display':'inline-block', 'margin':'0px 20px 0px 20px'}),   
 html.Span(
  children = [
  f"Prepared on {(datetime.now().date()).strftime('%B %d, %Y')}",
  html.Br(),
  'by ', html.B('Bayan Saparbayeva'),
  html.Br(),
  html.I('Data Scientist')],
  style={'text-align':'center', 'font-size':22, 'display':'inline-block'}),
 html.Img(src=src4, style={'width':'150px', 'display':'inline-block', 'margin':'0px 20px 0px 20px'})],
 style={'text-align':'center', 'display':'inline-block', 'width':'100%',
        'background-color':'black', 'color':'deeppink'})

@app.callback(
 Output(component_id='information', component_property='children'),
 Input(component_id='ticker', component_property='value')
)

def update_informatiol(tick):
    ticker='ELY'
    if tick: ticker=tick
    
    information_text = si.get_company_info(ticker).loc['longBusinessSummary', 'Value']
    return information_text

@app.callback(
 Output(component_id='column_y', component_property='options'),
 Output(component_id='column_c', component_property='options'),
 Output(component_id='column_1', component_property='options'),   
 Output(component_id='column_r', component_property='options'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value')    
)

def update_columns(tick, per):
    ticker = 'ELY'
    if tick: ticker=tick
    
    fs = get_yearly_fs(ticker)
    if per=='quarterly': fs = get_quarterly_fs(ticker)
        
    columns1 = fs.columns
    
    del columns1[columns1.index('date')]
    
    column_options = [{'label':x, 'value':x} for x in fs.columns]
    
    positive_columns = []

    for col in columns1:
        if (fs[col] > 0.1).all(): positive_columns.append(col)
            
    column_r_options = [{'label':x, 'value':x} for x in positive_columns]
    
    return column_options, column_options, column_options, column_r_options

@app.callback(
 Output(component_id='scatter_fig', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),
 Input(component_id='column_y', component_property='value'),
 Input(component_id='column_c', component_property='value'),
 Input(component_id='column_r', component_property='value')   
)

def update_plot2(tick, per, column_name_y, column_name_c, column_name_r):
    ticker = 'ELY'
    if tick: ticker=tick
        
    col_name_y = 'totalAssets'
    if column_name_y: col_name_y = column_name_y
        
    col_name_c = 'totalLiab'
    if column_name_c: col_name_c = column_name_c

    col_name_r = 'totalStockholderEquity'
    if column_name_r: col_name_r = column_name_r
        
    fs = get_yearly_fs(ticker)
    if per=='quarterly': fs = get_quarterly_fs(ticker)
        
    scatter_fig = px.scatter(fs, x='date', y=col_name_y, size=col_name_r, color=col_name_c,
            custom_data=['date'], size_max=50, height=360,
            title=get_title(ticker))
    scatter_fig.update_layout(plot_style())
 
    return scatter_fig

@app.callback(
 Output(component_id='bar_fig1', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value')
)

def update_plot1(tick, per):
    ticker = 'ELY'
    if tick: ticker=tick
    
    fs = get_yearly_fs(ticker)
    if per=='quarterly': fs = get_quarterly_fs(ticker)
        
    fs_melt = pd.melt(fs, id_vars=['date'])    
    
    #fs_melt['date'] = fs_melt['date'].astype('str')
    fs_melt['value'] = fs_melt['value'].astype('float')
    
    #fs_melt['title']=ticker
        
    #line_polar_fig = px.line_polar(fs_melt[fs_melt.Breakdown.isin([col_name_1, col_name_2, col_name_3])], 
    #    r='value', theta='Breakdown', color='date', line_close=True,
    #    title=get_title(ticker))
    #line_polar_fig.update_layout(plot_style())
        
    bar_fig1 = px.bar(fs_melt[fs_melt.Breakdown.isin(['inventory',
        'cash', 'totalRevenue', 'ebit', 'shortLongTermDebt'])], 
                      height=330,
        y='value', color='Breakdown', x='date', barmode="group", custom_data=['date'])
    bar_fig1.update_layout({'paper_bgcolor':'black', 'plot_bgcolor':'rgb(82, 82, 82)',
                               'font': {'color':'violet'}})
 
    return bar_fig1

@app.callback(
 Output(component_id='bar_fig2', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),
 Input(component_id='bar_fig1', component_property='clickData')   
)

def update_plot3(tick, per, clickData):
    ticker = 'ELY'
    if tick: ticker=tick
    
    fs = get_yearly_fs(ticker)
    if per=='quarterly': fs = get_quarterly_fs(ticker)
        
    cdate = fs.index[0]
    if clickData: cdate = pd.to_datetime(clickData['points'][0]['customdata'][0])
        
    #ser = fs.loc[fs.date[0]][:-1].astype('float')    
        
    ser = fs.loc[cdate, :-1].astype('float')
        
    bar_fig = px.bar(ser, height=600, title=get_title(ticker))
    bar_fig.update_layout(plot_style())
 
    return bar_fig

@app.callback(
 Output(component_id='column_2', component_property='options'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),
 Input(component_id='column_1', component_property='value')   
)

def update_columns2(tick, per, column_name_1):
    ticker = 'ELY'
    if tick: ticker=tick
    
    fs = get_yearly_fs(ticker)
    if per=='quarterly': fs = get_quarterly_fs(ticker)
        
    col_name_1 = 'totalAssets'
    if column_name_1: col_name_1 = column_name_1
    
    columns2 = list(fs.columns)
    del columns2[columns2.index('date')]
    del columns2[columns2.index(col_name_1)]
    
            
    column_2_options = [{'label':x, 'value':x} for x in columns2]
    
    return column_2_options

@app.callback(
 Output(component_id='column_3', component_property='options'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),
 Input(component_id='column_1', component_property='value'),
 Input(component_id='column_2', component_property='value')   
)

def update_columns3(tick, per, column_name_1, column_name_2):
    ticker = 'ELY'
    if tick: ticker=tick
    
    fs = get_yearly_fs(ticker)
    if per=='quarterly': fs = get_quarterly_fs(ticker)
        
    col_name_1 = 'totalAssets'
    if column_name_1: col_name_1 = column_name_1
        
    col_name_2 = 'totalLiab'
    if column_name_2: col_name_2 = column_name_2
    
    columns3 = list(fs.columns)
    del columns3[columns3.index('date')]
    del columns3[columns3.index(col_name_1)]
    del columns3[columns3.index(col_name_2)]
    
            
    column_3_options = [{'label':x, 'value':x} for x in columns3]
    
    return column_3_options

@app.callback(
 Output(component_id='line_polar_fig', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),   
 Input(component_id='column_1', component_property='value'),
 Input(component_id='column_2', component_property='value'),
 Input(component_id='column_3', component_property='value')   
)

def update_plot4(tick, per, column_name_1, column_name_2, column_name_3):
    ticker = 'ELY'
    if tick: ticker=tick
        
    col_name_1 = 'totalAssets'
    if column_name_1: col_name_1 = column_name_1
        
    col_name_2 = 'totalLiab'
    if column_name_2: col_name_2 = column_name_2

    col_name_3 = 'totalStockholderEquity'
    if column_name_3: col_name_3 = column_name_3
        
    fs = get_yearly_fs(ticker)
    if per=='quarterly': fs = get_quarterly_fs(ticker)
    
    fs_melt = pd.melt(fs, id_vars=['date'])
    
    fs_melt['date'] = fs_melt['date'].astype('str')
    fs_melt['value'] = fs_melt['value'].astype('float')
        
    line_polar_fig = px.line_polar(fs_melt[fs_melt.Breakdown.isin([col_name_1, col_name_2, col_name_3])], 
        r='value', theta='Breakdown', color='date', line_close=True,
        title=get_title(ticker))
    line_polar_fig.update_layout(plot_style())
 
    return line_polar_fig

if __name__ == '__main__':
    app.run_server(port=8030, debug=True)